In [1]:
import pandas as pd
import numpy as np
import os
import sys

## merge_df.txt and rename_df.txt

In [ ]:
# # load data
# with open("/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/sample_list.txt", "r") as file:
#     sample_list = [samp.strip() for samp in file.readlines()]
# with open("/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/accession_list.txt", "r") as file:
#     accession_list = [samp.strip() for samp in file.readlines()]


In [ ]:
# sample_to_accessions_map = {
#     sample: [acc for acc in accession_list if sample in acc][0]
#     for sample in sample_list
# }

In [ ]:
# fastq_folder_dir = "/u/project/ngarud/Garud_lab/ConventionalMouse/fastqs/download_2/01.RawData/"

# accession_folders = [name for name in os.listdir(fastq_folder_dir) 
#          if os.path.isdir(os.path.join(fastq_folder_dir, name))]

# duplicated_samples = []
# for accession_folder in accession_folders:
#     accession_path = "%s%s/" % (fastq_folder_dir, accession_folder)
#     fastq_pairs = len([file_name for file_name in os.listdir(accession_path) if "_1.fq.gz" in file_name])
#     if fastq_pairs >= 2:
#         duplicated_samples.append([file_name for file_name in os.listdir(accession_path) if "_1.fq.gz" in file_name][0][:7])


In [ ]:
# # rename df
# rename_samples = [samp for samp in sample_list if samp not in duplicated_samples]
# corresponding_accessions = [sample_to_accessions_map[samp] for samp in rename_samples]
# rename_df = pd.DataFrame({"run_accession": corresponding_accessions,"new_sample_name": rename_samples})


In [ ]:
# # merge df
# fastq_pairs_vec = []
# corresponding_samples_vec = []
# merge_groups_vec = []

# for i,sample in enumerate(duplicated_samples):
#     accession_path = "%s%s/" % (fastq_folder_dir, sample)
#     fastq_pairs = [file_name[:-8] for file_name in os.listdir(accession_path) if "_1.fq.gz" in file_name]
#     corresponding_samples = [sample]*len(fastq_pairs)
#     merge_groups = [i+1]*len(fastq_pairs)
    
#     fastq_pairs_vec = fastq_pairs_vec + fastq_pairs
#     corresponding_samples_vec = corresponding_samples_vec + corresponding_samples
#     merge_groups_vec = merge_groups_vec + merge_groups
    

# merge_df = pd.DataFrame({"run_accession": fastq_pairs_vec,
#                          "new_sample_name": corresponding_samples_vec,
#                          "merge_no": merge_groups_vec})

    

In [ ]:
# # Saving
# ## rename_df 
# out_path = "/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/rename_df.txt"
# rename_df.to_csv(out_path, sep = ",", index=False)

# ## rename_df 
# out_path = "/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/merge_df.txt"
# merge_df.to_csv(out_path, sep = ",", index=False)

In [ ]:
# # list of paths
# fastq_folder_fir = "/u/project/ngarud/Garud_lab/ConventionalMouse/fastqs/download_2/01.RawData/"
# subdirs = [name for name in os.listdir(fastq_folder_fir) 
#            if os.path.isdir(os.path.join(fastq_folder_fir, name))]


## mapping rates

In [45]:
# load the data
read_counts_df = pd.read_csv("/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/read_counts.txt", delim_whitespace=True).rename(columns = {"accession":"sample"})
sample_list = read_counts_df['sample'].tolist()


In [54]:
snps_summary_all = {"sample": [], 
                    "mapped_reads": [],
                    "species_count": []}

for sample_ in sample_list:
    try:
        snps_summary_path = "/u/project/ngarud/Garud_lab/ConventionalMouse/midas_output/%s/snps/snps_summary.tsv" % (sample_)
        snps_summary = pd.read_csv(snps_summary_path, sep = "\t")
        snps_summary_all['sample'].append(sample_)
        snps_summary_all['mapped_reads'].append(snps_summary.mapped_reads.sum()/2)
        snps_summary_all['species_count'].append(len(snps_summary.species_id.unique()))

    except FileNotFoundError:
        sys.stderr.write("SNPs summary didn't exist for %s\n" % (sample_))
        snps_summary_all['sample'].append(sample_)
        snps_summary_all['mapped_reads'].append(0.0)
        snps_summary_all['species_count'].append(0.0)
    except:
        sys.stderr.write("Something else went wrong for %s\n" % (sample_))
        snps_summary_all['sample'].append(sample_)
        snps_summary_all['mapped_reads'].append(0.0)
        snps_summary_all['species_count'].append(0.0)

snps_summary_all = pd.DataFrame(snps_summary_all)

SNPs summary didn't exist for Duo_1_1
SNPs summary didn't exist for Duo_1_3
SNPs summary didn't exist for Duo_2_1
SNPs summary didn't exist for Duo_2_3
SNPs summary didn't exist for Duo_3_3
SNPs summary didn't exist for Jej_2_2


In [64]:
mapping_summary_df = pd.merge(snps_summary_all, read_counts_df, on = "sample")
mapping_summary_df['mapping_rate'] = (mapping_summary_df.mapped_reads/mapping_summary_df.read_count)*100
mapping_summary_df = mapping_summary_df[['sample','species_count','read_count','mapped_reads','mapping_rate']]
mapping_summary_df = mapping_summary_df.sort_values(by = 'species_count', ascending = False)
mapping_summary_df = mapping_summary_df.reset_index(drop=True)


In [65]:
out_path = "/u/project/ngarud/michaelw/Diversity-Along-Gut/ConventionalMouse/metadata/mapping_rate_summary.csv"
mapping_summary_df.to_csv(out_path, sep = ",", index=False)